## 补充
* 权值太多，参数过多，需要样本数量大
* 引入局部感受野

###  卷积层就是对图像像素进行特征提取，多个不同卷积核产生不同特征图，增加深度，减少参数
### 池化层就是降低分辨率，缩小尺寸，减少参数，加快计算

### CONV(卷积层-RELU(激活函数)-POOL(池化层)... FC(全连接层) 如下图

![](https://cdn.nlark.com/yuque/0/2019/png/232596/1572371566724-41591b3b-8b49-454d-a4df-d2e3ba33e6cc.png)

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# 载入数据集
mnist = input_data.read_data_sets("../MNIST_data", one_hot=True) 


Extracting ../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data\t10k-labels-idx1-ubyte.gz


In [ ]:
batch_size = 100  # 一个批次的大小，可优化
n_batch = mnist.train.num_examples // batch_size # 一共有多少个批次

# 权值初始化
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

# 偏置值初始化
def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))
# 卷积层
def conv2d(x,W):
    # x表示输入,是一个张量 [batch, in_height, in_width, in_channels(通道数 1 | 3)]
    # W表示过滤器filter，即卷积核kernel [filter_height, filter_width, in_channels, out_channels]
    # strides表示步长 strides = [1, stride, stride, 1].strides[1]和[2],分别代表x方向步长和y方向
    # padding表示是否补零   string "SAME" or "VALID"
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME') # 步长。

# 池化层
def max_pool_2x2(x):
    #(kernel size) ksize = [1,x,y,1] x,y代表窗口的大小 可选参数length 1, 2 or 4
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') # 窗口2x2

# 定义两个占位符（训练数据集和标签）
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

# 将x转化为4d向量[batch,height,width,channel]   x是四维的，我们只需要对二三维卷积
x_image = tf.reshape(x,[-1,28,28,1]) # -1表任意batch

# 初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征  生成32个5x5卷积核
b_conv1 = bias_vairable([32]) #每个卷积核一个偏置值

# 28*28*1 的图片卷积之后变为28*28*32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # 池化之后变为 14*14*32

# 初始化第二个卷积层的权重和偏置 第二次卷积之后变为 14*14*64
W_conv2 = weight_variable([5,5,32,64]) # 5*5的采样窗口，64个卷积核从32个平面抽取特征
b_conv2 = bias_vairable([64]) # 每一个卷积核一个偏执值
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
# 第二次池化之后变为 7*7*64
h_pool2 = max_pool_2x2(h_conv2)

# 28*28的图像第一次卷积后还是28*28,第一次池化后变成14*14，最后得到32张14*14的平面
# 第二次卷积后为14*14，第二次池化后为7*7，最后得到64张7*7的平面
# 经过上面的操作后得到64张7*7的平面

# 第一个全连接层
W_fc1 = weight_variable([7*7*64,1024]) # 上一层有7*7*64个神经元，全连接成有1024个神经元
b_fc1 = bias_vairable([1024]) # 1024个节点
# 7*7*64的图像变成1维向量 #把池化层后的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) # #求第一个全连接层的输出
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_vairable([10])
logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
prediction = tf.nn.sigmoid(logits)
# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
# 使用AAdam进行优化
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

prediction_2 = tf.nn.softmax(prediction) # softmax
# 比较概率最大的标签是否相同，结果存放在一个布尔型列表中
correct_prediction = (tf.equal(tf.argmax(prediction_2,1), tf.argmax(y,1)))
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("Iter: " + str(epoch) + ", acc: " + str(acc))